In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [2]:
main_path = r'd:\Documents\Private\Kaggle\IEEEFraud'

In [3]:
import sys
sys.path.append(main_path)
from BayDS import *

In [4]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [5]:
train = pd.read_pickle(f'{main_learning_folder}/train.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test.pkl')


In [6]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud'].astype(np.uint8)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

X.drop('Date', axis=1, inplace=True)
X=X.astype(np.float32)
test.drop('Date', axis=1, inplace=True)
test=test.astype(np.float32)
del train
gc.collect()

20

In [7]:
# Setting model_folder
model_name = 'catboost-test'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [8]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [36]:
params = {'grow_policy':'SymmetricTree',
#     'max_leaves': 10,
#           'min_child_weight': 0.03454472573214212,
#           'feature_fraction': 0.3797454081646243,
#           'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'CrossEntropy',
#           'max_depth': -1,
          'learning_rate': 0.03,
#           "boosting_type": "gbdt",
#           "bagging_seed": 11,
          "custom_metric": 'AUC',
          "logging_level": 'Verbose',
#           'reg_alpha': 0.3899927210061127,
#           'reg_lambda': 0.6485237330340494,
          'random_state': 47,
          'bootstrap_type': 'Bayesian',
          'task_type': 'GPU'
         }
train_options = {
    "model_type":'cat',
    "params": params,
    "eval_metric":'auc',
    'early_stopping_rounds': 200,
    'n_estimators': 50,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
    
}

In [10]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)

In [37]:
print(gc.collect())
result_dict_lgb = train_model_classification(X=X, X_test=test, y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=None)

5953
Fold 1 started at Sat Aug 31 17:53:24 2019
Fold 2 started at Sat Aug 31 17:54:30 2019
Fold 3 started at Sat Aug 31 17:54:46 2019
Fold 4 started at Sat Aug 31 17:55:02 2019
Fold 5 started at Sat Aug 31 17:55:18 2019
CV mean score: 0.6116, std: 0.0055.


In [41]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_lgb,f)
#     f.write(q)